In [1]:
import pandas as pd
import numpy as np
from xml.etree import cElementTree as ElementTree
import xml.etree.ElementTree as et
from tqdm import tqdm
import logging
import re
import requests
#exec(requests.get('https://raw.githubusercontent.com/ma-ji/npo_classifier/master/API/npoclass.py').text)

#logging.basicConfig(format='%(asctime)s: %(message)s', filename='analysis.log', level=logging.DEBUG)

# Data Cleaning

In [29]:
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 300)
pd.set_option('display.min_rows', 200)
df2016 = pd.read_csv("C:/Users/tenis/OneDrive/Desktop/Data_Projects/Impact_Capital/data/interim/2016_objects.csv")
df2017 = pd.read_csv("C:/Users/tenis/OneDrive/Desktop/Data_Projects/Impact_Capital/data/interim/2017_objects.csv")
df = pd.read_csv("C:/Users/tenis/OneDrive/Desktop/Data_Projects/Impact_Capital/data/interim/form_990_objects_2018_2020.csv")

df = df.append(df2017, ignore_index = True)
df = df.append(df2016, ignore_index = True)

#Fixing Types of Orgs 
df.loc[df['TypeOfOrganizationCorpInd'] != "X", 'TypeOfOrganizationCorpInd'] = 0
df.loc[df['TypeOfOrganizationCorpInd'] == "X", 'TypeOfOrganizationCorpInd'] = 1
df.loc[df['TypeOfOrganizationTrustInd'] != "X", 'TypeOfOrganizationTrustInd'] = 0
df.loc[df['TypeOfOrganizationTrustInd'] == "X", 'TypeOfOrganizationTrustInd'] = 1
df.loc[df['TypeOfOrganizationTrustInd'] != "X", 'TypeOfOrganizationTrustInd'] = 0
df.loc[df['TypeOfOrganizationTrustInd'] == "X", 'TypeOfOrganizationTrustInd'] = 1
df.loc[df['TypeOfOrganizationAssocInd'] != "X", 'TypeOfOrganizationAssocInd'] = 0
df.loc[df['TypeOfOrganizationAssocInd'] == "X", 'TypeOfOrganizationAssocInd'] = 1
df.loc[df['TypeOfOrganizationOtherInd'] != "X", 'TypeOfOrganizationOtherInd'] = 0
df.loc[df['TypeOfOrganizationOtherInd'] == "X", 'TypeOfOrganizationOtherInd'] = 1

#SchoolOperatingInd	OperateHospitalInd	AddressChangeInd	InitialReturnInd	AmendedReturnInd	TerminateOperationsInd
df[['SchoolOperatingInd', 'OperateHospitalInd','AddressChangeInd',
    'InitialReturnInd', 'AmendedReturnInd','FinalReturnInd','TerminateOperationsInd','TotalFunctionExpenses_FundraisingAmt']] =df[['SchoolOperatingInd', 'OperateHospitalInd','AddressChangeInd','InitialReturnInd',
                 'AmendedReturnInd','FinalReturnInd','TerminateOperationsInd', 'TotalFunctionExpenses_FundraisingAmt']].fillna(0)

df.loc[df['SchoolOperatingInd'] == 'false', 'SchoolOperatingInd'] = 0
df.loc[df['OperateHospitalInd'] == 'false', 'OperateHospitalInd'] = 0
df.loc[df['AddressChangeInd'] == 'false', 'AddressChangeInd'] = 0
df.loc[df['InitialReturnInd'] == 'false', 'InitialReturnInd'] = 0
df.loc[df['AmendedReturnInd'] == 'false', 'AmendedReturnInd'] = 0
df.loc[df['FinalReturnInd'] == 'false', 'FinalReturnInd'] = 0
df.loc[df['TerminateOperationsInd'] == 'false', 'TerminateOperationsInd'] = 0

df.loc[df['SchoolOperatingInd'] == '0', 'SchoolOperatingInd'] = 0
df.loc[df['OperateHospitalInd'] == '0', 'OperateHospitalInd'] = 0
df.loc[df['AddressChangeInd'] == '0', 'AddressChangeInd'] = 0
df.loc[df['InitialReturnInd'] == '0', 'InitialReturnInd'] = 0
df.loc[df['AmendedReturnInd'] == '0', 'AmendedReturnInd'] = 0
df.loc[df['FinalReturnInd'] == '0', 'FinalReturnInd'] = 0
df.loc[df['TerminateOperationsInd'] == '0', 'TerminateOperationsInd'] = 0

df.loc[df['SchoolOperatingInd'] != 0, 'SchoolOperatingInd'] = 1
df.loc[df['OperateHospitalInd'] != 0, 'OperateHospitalInd'] = 1
df.loc[df['AddressChangeInd'] != 0, 'AddressChangeInd'] = 1
df.loc[df['InitialReturnInd'] != 0, 'InitialReturnInd'] = 1
df.loc[df['AmendedReturnInd'] != 0, 'AmendedReturnInd'] = 1
df.loc[df['FinalReturnInd'] != 0, 'FinalReturnInd'] = 1
df.loc[df['TerminateOperationsInd'] != 0, 'TerminateOperationsInd'] = 1

#Merging the descriptions 
#The majority of the descriptions are just duplicates, so I'm deleting the ones that are exaclty the same 
#df.ActivityOrMissionDesc= df.ActivityOrMissionDesc.str.lower()
#df.MissionDesc= df.MissionDesc.str.lower()
#df.loc[df['ActivityOrMissionDesc'] ==  df['MissionDesc'], 'MissionDesc'] = ""

df = df.drop(['Unnamed: 0', 'Object_ID', 'MissionDesc', 'schedule0'], axis=1)


df['Ratio_Equity_Revenue'] = df['NetAssetsOrFundBalancesEOYAmt']/df['CYTotalRevenueAmt']

df['Ratio_AdminExp_Exp'] = df['TotalFunctionExpenses_ManagementAndGeneralAmt']/df['CYTotalExpensesAmt']

df['Operating_Margin'] = (df['CYTotalRevenueAmt'] - df['CYTotalExpensesAmt'])/df['CYTotalRevenueAmt']

df['Revenue_Concentration'] = (df['RelatedOrganizationsAmt']/df['CYTotalRevenueAmt'])**2 + (df['GovernmentGrantsAmt']/df['CYTotalRevenueAmt'])**2 \
                               + (df['FederatedCampaignsAmt']/df['CYTotalRevenueAmt'])**2 + (df['MembershipDuesAmt']/df['CYTotalRevenueAmt'])**2 \
                               + (df['FundraisingAmt']/df['CYTotalRevenueAmt'])**2 + (df['NoncashContributionsAmt']/df['CYTotalRevenueAmt'])**2 \
                               + (df['AllOtherContributionsAmt']/df['CYTotalRevenueAmt'])**2 + (df['TotalProgramServiceRevenueAmt']/df['CYTotalRevenueAmt'])**2 \
                               + (df['CYInvestmentIncomeAmt']/df['CYTotalRevenueAmt'])**2 + (df['CYOtherRevenueAmt']/df['CYTotalRevenueAmt'])**2

#Resource Dependence measures 
df['Dependence_Grants'] = df['GovernmentGrantsAmt']/df['CYTotalRevenueAmt']
df['Dependence_Campaigns'] = df['FederatedCampaignsAmt']/df['CYTotalRevenueAmt']
df['Dependence_Memberships'] = df['MembershipDuesAmt']/df['CYTotalRevenueAmt']
df['Dependence_Fundraising'] = df['FundraisingAmt']/df['CYTotalRevenueAmt']
df['Dependence_ProgramServ'] = df['TotalProgramServiceRevenueAmt']/df['CYTotalRevenueAmt']
df['Dependence_Invest'] = df['CYInvestmentIncomeAmt']/df['CYTotalRevenueAmt']

#Dependent Variable 
df['Ratio_ProgramExp_Rev'] = df['TotalFunctionExpenses_ProgramServicesAmt']/df['CYTotalRevenueAmt']


df['BusinessName'].fillna("none", inplace = True) 
df['ActivityOrMissionDesc'].fillna("none", inplace = True)
df['name_mission'] = df['BusinessName'] + ". " + df['ActivityOrMissionDesc']
df2015 = df[df['TaxYr'] == 2015]
df2015["FormationYr"].fillna(2014, inplace = True)

#Adding BERT data 
BERT = pd.read_csv("C:/Users/tenis/OneDrive/Desktop/Data_Projects/Impact_Capital/data/raw/BERT_classifications2015.csv")
df2015BERT = pd.concat([df2015.reset_index(drop=True), BERT], axis=1, ignore_index=False)
print("Number of nonprofits: ", len(df2015BERT))

df2015BERT = df2015BERT.drop(['Unnamed: 0'], axis=1)

C:\Users\tenis\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Number of nonprofits:  233520


In [30]:
#removing duplicates based on EIN. I need to go back and grab the timestamp, so that I remove the earlier submitted returns
df2015BERT = df2015BERT.drop_duplicates(subset="EIN")
print("Number of nonprofits: ", len(df2015BERT))

Number of nonprofits:  231523


### Ratios to measure nonprofit financial flexibility

* **Ratio of equity to total revenue** The implicit assumption is that organizations with a larger networth relative to revenue have a greater ability to replace revenue thanthose with a smaller or negative net worth. 
* **Ratio Administrative Expenses to total Expenses** Nonprofits withhigh administrative costs (relative to others in their category) are assumedto have the greatest opportunity to cut back on administrative costs without reducing programs. 

* **Low or Negative Operating Margins** A nonprofit’s operating margin is defined as its revenues less its expenditures, divided by its revenues. This shows the percentage that its net income represents of its revenues. 

* **Revenue Concentration** Closer to one signifies a nonprofit's revenue stream is concentrated in one resource

* **Resource Dependence Variables**  Dependence are funding source divided by total revenue

### Creating financially vulerable variable (Depdendent Variable)

* Reduction in the ratio of program expenses to total reveune 
* Hager, Galaskiewicz,Bielefeld, and Pins’s 1996 study found that many charities that ceased operations did so primarily for program-related reasons (suchas a reduction of donations or a decreased demand for program services). 


### NTEE Codes
I. Arts, Culture, and Humanities - A </br>
II. Education - B </br>
III. Environment and Animals - C, D </br>
IV. Health - E, F, G, H </br>
V. Human Services - I, J, K, L, M, N, O, P </br>
VI. International, Foreign Affairs - Q </br>
VII. Public, Societal Benefit - R, S, T, U, V, W </br>
VIII. Religion Related - X </br>

[NTEE Code Breakdown](https://nccs.urban.org/project/national-taxonomy-exempt-entities-ntee-codes#overview) </br>
[IRS Codes](https://nccs.urban.org/publication/irs-activity-codes)

In [54]:
pd.DataFrame(BERT.recommended.value_counts())

,recommended
V,83533
VII,39953
II,30610
IV,29726
I,18540
VIII,9467
III,8944
IX,8792
VI,3955


In [31]:
df2016 = df[df['TaxYr'] == 2016]
df2016["FormationYr"].fillna(2015, inplace = True)
df2017 = df[df['TaxYr'] == 2017]
df2018 = df[df['TaxYr'] == 2018]


df2016 = df2016[['EIN', 'Ratio_ProgramExp_Rev']]
df2017 = df2017[['EIN', 'Ratio_ProgramExp_Rev']]
df2018 = df2018[['EIN', 'Ratio_ProgramExp_Rev']]

df2016 = df2016.rename(columns={'Ratio_ProgramExp_Rev': 'Ratio_ProgramExp_Rev2016'})
df2017 = df2017.rename(columns={'Ratio_ProgramExp_Rev': 'Ratio_ProgramExp_Rev2017'})
df2018 = df2018.rename(columns={'Ratio_ProgramExp_Rev': 'Ratio_ProgramExp_Rev2018'})

df2015BERT = df2015BERT.merge(df2016, on='EIN')
df2015BERT = df2015BERT.merge(df2017, on='EIN')
df2015BERT = df2015BERT.merge(df2018, on='EIN')
print(len(df2015BERT))

#Dropping nonprofits that don't appear in 2017 or2018 is not very many. 
df2015BERT.dropna(subset = ["Ratio_ProgramExp_Rev2016"], inplace=True)
df2015BERT.dropna(subset = ["Ratio_ProgramExp_Rev2017"], inplace=True)
df2015BERT.dropna(subset = ["Ratio_ProgramExp_Rev2018"], inplace=True)
df2015BERT["Ratio_ProgramExp_Rev"].fillna(0, inplace = True) 
df2015BERT["Ratio_Equity_Revenue"].fillna(0, inplace = True) 
df2015BERT["Ratio_AdminExp_Exp"].fillna(0, inplace = True) 
df2015BERT["Operating_Margin"].fillna(0, inplace = True) 
df2015BERT["Revenue_Concentration"].fillna(0, inplace = True) 

df2015BERT['vulnerable2016'] = np.where(df2015BERT['Ratio_ProgramExp_Rev'] > df2015BERT['Ratio_ProgramExp_Rev2016'], 1, 0)
df2015BERT['vulnerable2017'] = np.where(df2015BERT['Ratio_ProgramExp_Rev2016'] > df2015BERT['Ratio_ProgramExp_Rev2017'], 1, 0)
df2015BERT['vulnerable2018'] = np.where(df2015BERT['Ratio_ProgramExp_Rev2017'] > df2015BERT['Ratio_ProgramExp_Rev2018'], 1, 0)
df2015BERT['vulnerable'] = np.where((df2015BERT['vulnerable2016'] == 1) & (df2015BERT['vulnerable2017'] == 1) & (df2015BERT['vulnerable2018'] == 1), 1, 0)

df2015BERT = df2015BERT.drop(['name_mission'], axis =1)
print("Number of nonprofits: ", len(df2015BERT))

167994
Number of nonprofits:  166996


In [32]:
df2015BERT = df2015BERT.drop_duplicates(subset="BusinessName")
print("Number of nonprofits: ", len(df2015BERT))
df2015BERT

Number of nonprofits:  157608


,TaxYr,TaxPeriodBeginDt,EIN,BusinessName,City,State,ZIPCd,FormationYr,LegalDomicileStateCd,RelatedOrganizationsAmt,GovernmentGrantsAmt,FederatedCampaignsAmt,MembershipDuesAmt,FundraisingAmt,NoncashContributionsAmt,AllOtherContributionsAmt,TotalContributionsAmt,TotalProgramServiceRevenueAmt,CYInvestmentIncomeAmt,CYOtherRevenueAmt,CYTotalRevenueAmt,CYTotalExpensesAmt,TotalFunctionExpenses_ProgramServicesAmt,TotalFunctionExpenses_ManagementAndGeneralAmt,CYRevenuesLessExpensesAmt,TotalAssetsEOYAmt,TotalLiabilitiesEOYAmt,NetAssetsOrFundBalancesEOYAmt,ActivityOrMissionDesc,TypeOfOrganizationCorpInd,TypeOfOrganizationTrustInd,TypeOfOrganizationAssocInd,TypeOfOrganizationOtherInd,SchoolOperatingInd,OperateHospitalInd,AddressChangeInd,InitialReturnInd,AmendedReturnInd,FinalReturnInd,TerminateOperationsInd,TotalFunctionExpenses_FundraisingAmt,Ratio_Equity_Revenue,Ratio_AdminExp_Exp,Operating_Margin,Revenue_Concentration,Dependence_Grants,Dependence_Campaigns,Dependence_Memberships,Dependence_Fundraising,Dependence_ProgramServ,Dependence_Invest,Ratio_ProgramExp_Rev,recommended,confidence,probabilities,Ratio_ProgramExp_Rev2016,Ratio_ProgramExp_Rev2017,Ratio_ProgramExp_Rev2018,vulnerable2016,vulnerable2017,vulnerable2018,vulnerable
0,2015,2015-01-01,221988373,POLISH CULTURAL FOUNDATION INC,CLARK,NJ,70661855.0,1973.0,NJ,0,0,0,146807,0,0,0,146807,261612,22763,260529,691711,555314,251108,304206,136397,2040863,189086,1851777,"TO PROVIDE EDUCATIONAL, CULTURAL AND CHARITABL...",1,0,0,0,0,0,0,0,0,0,0,0.0,2.677096e+00,0.547809,1.971878e-01,0.331031,0.000000,0.000000,0.212237,0.000000,0.378210,0.032908,0.363024,I,high (>=.99),"{'I': 0.9996774196624756, 'II': 0.928327500820...",0.487585,0.395017,0.481029,0,1,0,0
1,2015,2015-09-01,237124299,SPANISH PEOPLES COMMUNITY CENTER INC,LANDISVILLE,NJ,8326.0,1980.0,NJ,0,0,0,0,0,0,710842,710842,9743,7,10891,731483,846042,839931,6111,-114559,649207,106769,542438,TO PROMOTE THE UNITY OF SPANISH-SPEAKING PEOPL...,1,0,0,0,0,0,0,0,1,0,0,0.0,7.415593e-01,0.007223,-1.566120e-01,0.944759,0.000000,0.000000,0.000000,0.000000,0.013320,0.000010,1.148258,V,high (>=.99),"{'I': 0.9498765468597412, 'II': 0.808643996715...",1.051853,1.000352,0.978004,1,1,1,1
2,2015,2015-01-01,463198647,BIG HEART FOUNDATION INC,CLIFTON,NJ,7013.0,2013.0,DC,0,0,0,0,0,36735668,43591745,43591745,0,117,0,43591862,43405560,40582932,2822628,186302,418529,2555847,-2137318,"DELIVERY OF NON-FOOD ITEMS, FOOD BASKETS, HYGI...",1,0,0,0,0,0,0,0,1,0,0,0.0,-4.903021e-02,0.065029,4.273779e-03,1.710169,0.000000,0.000000,0.000000,0.000000,0.000000,0.000003,0.930975,IV,high (>=.99),"{'I': 0.4129273295402527, 'II': 0.446100682020...",0.846299,0.753361,0.771625,1,1,0,0
3,2015,2015-01-01,231371942,CHESTER COUNTY HISTORICAL SOCIETY,WEST CHESTER,PA,19380.0,2014.0,PA,0,1208980,0,61123,0,0,2702980,3973083,173219,147287,354122,4647711,1590276,936027,361554,3057435,13378885,1167740,12211145,TO PRESERVE THE HISTORY OF CHESTER COUNTY AND ...,1,0,0,0,0,0,0,0,1,0,0,0.0,2.627346e+00,0.227353,6.578367e-01,0.414262,0.260124,0.000000,0.013151,0.000000,0.037270,0.031690,0.201395,I,high (>=.99),"{'I': 0.9998829364776611, 'II': 0.644191145896...",1.018381,0.775317,1.000684,0,1,0,0
4,2015,2015-01-01,232120040,AMERICAN LEGION WESLEY DEHAVEN POST 364,LIVERPOOL,PA,17045.0,1955.0,PA,404,0,0,7518,0,0,40080,48002,0,22,4232,52256,67896,63713,3553,-15640,660862,405970,254892,PROVIDE SUPPORT FOR VETERANS AND EX VETERANS,1,0,0,0,0,0,0,0,0,0,0,0.0,4.877756e+00,0.052330,-2.992958e-01,0.615596,0.000000,0.000000,0.143869,0.000000,0.000000,0.000421,1.219248,VII,high (>=.99),"{'I': 0.5306817293167114, 'II': 0.531040549278...",0.452303,0.758374,0.000000,1,0,1,0
5,2015,2015-07-01,541555197,FAIRFAX COURT APPOINTED SPECIAL,FAIRFAX,VA,22030.0,1989.0,VA,74415,341014,29922,0,0,0,223751,669102,1435,53,37365,707955,708099,515584,165543,-144,403211,36826,366385,TO ADVOCATE FOR ABUSED AND NEGLECTED CHILDREN.,1,0,0,0,0,0,0,0,1,0,0,0.0,5.175258e-01,0.233785,-2.034028e-04,0.347538,0.481689,0.042265,0.000000,0.00000

# Removing outliers 

In [51]:
# Removing outliers from expenses because that is the focus of nonprofits

print("Before removing outliers","\n", df2015BERT['CYTotalExpensesAmt'].describe())

df2015 = df2015BERT[df2015BERT['CYTotalExpensesAmt'].between(df2015BERT['CYTotalExpensesAmt'].quantile(.01), df2015BERT['CYTotalExpensesAmt'].quantile(.99))] # without outliers

print("After removing outliers","\n", df2015['CYTotalExpensesAmt'].describe())

Before removing outliers 
 count    1.576080e+05
mean     9.147660e+06
std      1.653444e+08
min     -2.209473e+06
25%      1.929198e+05
50%      4.724060e+05
75%      1.700779e+06
max      4.825271e+10
Name: CYTotalExpensesAmt, dtype: float64
After removing outliers 
 count    1.544540e+05
mean     3.323318e+06
std      1.056868e+07
min      3.857000e+03
25%      1.965238e+05
50%      4.724060e+05
75%      1.643855e+06
max      1.307374e+08
Name: CYTotalExpensesAmt, dtype: float64


In [57]:
df2015BERT.to_csv(r"C:\Users\tenis\OneDrive\Desktop\Data_Projects\Impact_Capital\data\interim\IRS990_with_Bert.csv")

In [57]:
df2015ArtsCulture = df2015[df2015['recommended'] == "I"]
df2015Education = df2015[df2015['recommended'] == "II"]
df2015Enviroment_Animals = df2015[df2015['recommended'] == "III"]
df2015Health = df2015[df2015['recommended'] == "IV"]
df2015HumanServices = df2015[df2015['recommended'] == "V"]
df2015International = df2015[df2015['recommended'] == "VI"]
df2015SocietalBenefit = df2015[df2015['recommended'] == "VII"]
df2015Religion = df2015[df2015['recommended'] == "VIII"]


In [55]:
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


def logfunc(data):
    "data equals the dataframe to be used."
    logistic = data[['FormationYr',
                   'CYTotalExpensesAmt',
                   'Ratio_Equity_Revenue',
                   'Ratio_AdminExp_Exp',
                   'Operating_Margin',
                   'Revenue_Concentration',
                   'Ratio_ProgramExp_Rev',
                   'vulnerable'
                     ]]

    #'Dependence_Grants',
    #'Dependence_Campaigns',
    #'Dependence_Memberships',
    #'Dependence_Fundraising', 
    #'Dependence_ProgramServ',
    #'Dependence_Invest'

    #Apparently no trust in this dataset. 
    logistic = logistic.replace([np.inf, -np.inf], np.nan)
    logistic = logistic.fillna(0)



    logistic['vulnerable'] = logistic['vulnerable'].astype(float)



    X = logistic.loc[:, logistic.columns != 'vulnerable']
    X_colnames = X.columns


    X = pd.DataFrame(X, columns = X_colnames)
    y = logistic.loc[:, logistic.columns == 'vulnerable']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


    y_train=y_train.values.reshape(-1,1)

    logit_model=sm.Logit(y_train,X_train)
    result=logit_model.fit(max_iter=1000)
    print(result.summary2())
    
    return

# Arts and Culture

In [58]:
logfunc(df2015ArtsCulture)

Optimization terminated successfully.
         Current function value: 0.213488
         Iterations 18
                            Results: Logit
Model:                 Logit             Pseudo R-squared:  0.013     
Dependent Variable:    y                 AIC:               3678.3156 
Date:                  2020-11-10 11:32  BIC:               3727.7176 
No. Observations:      8582              Log-Likelihood:    -1832.2   
Df Model:              6                 LL-Null:           -1856.1   
Df Residuals:          8575              LLR p-value:       1.2323e-08
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        18.0000                                        
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
FormationYr           -0.0012   0.0001 -16.9913 0.0000 -0.0013 -0.0011
CY

# Education 

In [59]:
logfunc(df2015Education)

Optimization terminated successfully.
         Current function value: 0.223601
         Iterations 17
                            Results: Logit
Model:                 Logit             Pseudo R-squared:  0.004     
Dependent Variable:    y                 AIC:               5709.1188 
Date:                  2020-11-10 11:34  BIC:               5761.2835 
No. Observations:      12735             Log-Likelihood:    -2847.6   
Df Model:              6                 LL-Null:           -2859.8   
Df Residuals:          12728             LLR p-value:       0.00042467
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        17.0000                                        
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
FormationYr           -0.0014   0.0000 -36.0806 0.0000 -0.0014 -0.0013
CY

# Environment and Animals

In [60]:
logfunc(df2015Enviroment_Animals)

Optimization terminated successfully.
         Current function value: 0.214307
         Iterations 10
                            Results: Logit
Model:                Logit              Pseudo R-squared:   0.008    
Dependent Variable:   y                  AIC:                1962.9064
Date:                 2020-11-10 11:36   BIC:                2007.8619
No. Observations:     4547               Log-Likelihood:     -974.45  
Df Model:             6                  LL-Null:            -982.36  
Df Residuals:         4540               LLR p-value:        0.014827 
Converged:            1.0000             Scale:              1.0000   
No. Iterations:       10.0000                                         
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
FormationYr           -0.0012   0.0001 -12.5204 0.0000 -0.0014 -0.0010
CY

# Health Organizations

In [61]:
logfunc(df2015Health)

Optimization terminated successfully.
         Current function value: 0.227014
         Iterations 13
                            Results: Logit
Model:                 Logit             Pseudo R-squared:  0.006     
Dependent Variable:    y                 AIC:               5894.5616 
Date:                  2020-11-10 11:36  BIC:               5946.8446 
No. Observations:      12952             Log-Likelihood:    -2940.3   
Df Model:              6                 LL-Null:           -2958.6   
Df Residuals:          12945             LLR p-value:       2.1054e-06
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        13.0000                                        
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
FormationYr           -0.0014   0.0000 -34.7980 0.0000 -0.0014 -0.0013
CY

# Human Services

In [63]:
logfunc(df2015HumanServices)

Optimization terminated successfully.
         Current function value: 0.224002
         Iterations 10
                            Results: Logit
Model:                 Logit             Pseudo R-squared:  0.000     
Dependent Variable:    y                 AIC:               17978.9226
Date:                  2020-11-10 11:37  BIC:               18039.1165
No. Observations:      40100             Log-Likelihood:    -8982.5   
Df Model:              6                 LL-Null:           -8985.4   
Df Residuals:          40093             LLR p-value:       0.42774   
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        10.0000                                        
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
FormationYr           -0.0014   0.0000 -97.8401 0.0000 -0.0014 -0.0013
CY

# International

In [64]:
logfunc(df2015International)

Optimization terminated successfully.
         Current function value: 0.172175
         Iterations 14
                            Results: Logit
Model:                 Logit              Pseudo R-squared:   0.014   
Dependent Variable:    y                  AIC:                651.7366
Date:                  2020-11-10 11:39   BIC:                690.4048
No. Observations:      1852               Log-Likelihood:     -318.87 
Df Model:              6                  LL-Null:            -323.38 
Df Residuals:          1845               LLR p-value:        0.17179 
Converged:             1.0000             Scale:              1.0000  
No. Iterations:        14.0000                                        
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
FormationYr           -0.0016   0.0001 -18.4366 0.0000 -0.0018 -0.0014
CY

# Societal Benefit

In [65]:
logfunc(df2015SocietalBenefit)

Optimization terminated successfully.
         Current function value: 0.192274
         Iterations 13
                            Results: Logit
Model:                 Logit             Pseudo R-squared:  0.004     
Dependent Variable:    y                 AIC:               7319.2570 
Date:                  2020-11-10 11:40  BIC:               7374.2213 
No. Observations:      18997             Log-Likelihood:    -3652.6   
Df Model:              6                 LL-Null:           -3667.9   
Df Residuals:          18990             LLR p-value:       3.0999e-05
Converged:             1.0000            Scale:             1.0000    
No. Iterations:        13.0000                                        
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
FormationYr           -0.0016   0.0000 -55.6633 0.0000 -0.0016 -0.0015
CY

# Religion 

In [66]:
logfunc(df2015Religion)

Optimization terminated successfully.
         Current function value: 0.221039
         Iterations 16
                            Results: Logit
Model:                Logit              Pseudo R-squared:   0.009    
Dependent Variable:   y                  AIC:                1999.8149
Date:                 2020-11-10 11:41   BIC:                2044.6853
No. Observations:     4492               Log-Likelihood:     -992.91  
Df Model:             6                  LL-Null:            -1001.5  
Df Residuals:         4485               LLR p-value:        0.0085342
Converged:            1.0000             Scale:              1.0000   
No. Iterations:       16.0000                                         
----------------------------------------------------------------------
                       Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
----------------------------------------------------------------------
FormationYr           -0.0014   0.0000 -30.2067 0.0000 -0.0015 -0.0013
CY

In [52]:
NTEE_Code = pd.get_dummies(df2015.recommended, prefix='NTEE_Code')
df2015_all_categories= pd.concat([df2015.reset_index(drop=True), NTEE_Code], axis=1, ignore_index=False)

In [54]:
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

logistic = df2015_all_categories[['FormationYr',
               'CYTotalExpensesAmt',
               'Ratio_Equity_Revenue',
               'Ratio_AdminExp_Exp',
               'Operating_Margin',
               'Revenue_Concentration',
               'Ratio_ProgramExp_Rev',
               'vulnerable',
               'NTEE_Code_I',
               'NTEE_Code_II',
               'NTEE_Code_III',
               'NTEE_Code_IV',
               'NTEE_Code_IX',
               'NTEE_Code_V',
               'NTEE_Code_VI',
               'NTEE_Code_VII',
               'NTEE_Code_VIII'
                 ]]

#'Dependence_Grants',
#'Dependence_Campaigns',
#'Dependence_Memberships',
#'Dependence_Fundraising', 
#'Dependence_ProgramServ',
#'Dependence_Invest'

#Apparently no trust in this dataset. 
logistic = logistic.replace([np.inf, -np.inf], np.nan)
logistic = logistic.fillna(0)



logistic['vulnerable'] = logistic['vulnerable'].astype(float)



X = logistic.loc[:, logistic.columns != 'vulnerable']
X_colnames = X.columns

#normalizeing
#X = preprocessing.normalize(X)
X = pd.DataFrame(X, columns = X_colnames)
y = logistic.loc[:, logistic.columns == 'vulnerable']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


y_train=y_train.values.reshape(-1,1)

logit_model=sm.Logit(y_train,X_train)
result=logit_model.fit(max_iter=500)
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.249219
         Iterations 10
                            Results: Logit
Model:                Logit              Pseudo R-squared:   -0.230    
Dependent Variable:   y                  AIC:                58190.1743
Date:                 2020-11-10 11:22   BIC:                58257.8468
No. Observations:     116717             Log-Likelihood:     -29088.   
Df Model:             6                  LL-Null:            -23651.   
Df Residuals:         116710             LLR p-value:        1.0000    
Converged:            1.0000             Scale:              1.0000    
No. Iterations:       10.0000                                          
-----------------------------------------------------------------------
                       Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------------
FormationYr           -0.0014   0.0000 -159.6909 0.0000 -0.001